In [9]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

In [29]:
K = np.complex(250,10)
k = np.complex(15,2)
E_p = 4*K

def create_cat_state():
    N = 10
    a = destroy(10)
    a_dag = create(10)
    
    r_not = (((4*E_p*E_p) - ((k*k)/4))/(4*K*K))**(1/4)
    theta_not = 0.5*np.arctan((k/(((16*E_p*E_p) - k*k)**(0.5))))
    a_not = r_not * np.exp(np.complex(0,theta_not))
    
    H_not = (-1)*K*a_dag*a_dag*a*a + (E_p*a_dag*a_dag + E_p.conjugate()*a*a)
    H_eff = H_not - np.complex(0,1)*(k*a_dag*a*0.5)
    D = displace(N,a_not)
    
    H_eff_prime = D.dag()*H_eff*D
    s = coherent(N,a_not)
    r = mesolve(H, psi0, tlist, [], [])
    return H_eff_prime

In [32]:
#create_cat_state()

In [30]:
def plot_wigner(r):
    figure, axis = plt.subplots(1, 1, figsize=(8,8))
    x = np.linspace(-7.5,7.5,200)
    W = wigner(r, x, x)
    limit = abs(W).max()
    axis.contourf(x, x, W, 1000, norm=mpl.colors.Normalize(-limit,limit), cmap=mpl.cm.get_cmap('RdBu'))
    print(abs(W).max())
    return figure, axis


In [31]:
#plot_wigner(create_cat_state())

In [28]:
def plot_fock_distribution_vs_time(tlist, states, fig=None, ax=None):
    
    Z = np.zeros((len(tlist), states[0].shape[0]))
    
    for state_idx, state in enumerate(states):
        Z[state_idx,:] = real(ket2dm(state).diag())
        
    if fig is None or axes is None:
        fig, ax = plt.subplots(1, 1, figsize=(8,6))

    Y, X = meshgrid(tlist, range(states[0].shape[0]))
    p = ax.pcolor(X, Y, Z.T, norm=mpl.colors.Normalize(0, 0.5), cmap=mpl.cm.get_cmap('Reds'), edgecolors='k')
    ax.set_xlabel(r'$N$', fontsize=16)
    ax.set_ylabel(r'$t$', fontsize=16)    
    
    cb = fig.colorbar(p)
    cb.set_label('Probability')
    
    return fig, ax